# RESNET 18

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as vF
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import v2
from torchvision.models import mobilenet_v2, mobilenet_v3_large, vgg19, vgg16, resnet50, resnet18
from timm.models.swin_transformer import SwinTransformer
from sklearn.linear_model import LinearRegression
import kagglehub
import numpy as np
import os
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
from losses import SupConLoss
from torch.nn import TripletMarginLoss
import random
from Utils import GetDataset

c:\Users\tsope\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Hyperparameters
batch_size  = 128
epochs      = 400
lr          = 1e-4
split_ratio = 0.9   # %
device      = "cuda:0"
features_dim = 64

dataset = GetDataset(data_augmentation=False)

# Get the total size of the dataset
dataset_size = len(dataset)

# Calculate sizes for training and validation splits
train_size = int(split_ratio * dataset_size)
val_size = dataset_size - train_size

# Perform the split
generator1 = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator1)

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True )
val_loader   = DataLoader(val_dataset  , batch_size=batch_size, shuffle=False)

class ResNet18E(nn.Module):
  def __init__(self, device : str = "cpu")->None:
    super(ResNet18E, self).__init__()
    self.backbone = resnet18(pretrained=True)
    self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.backbone.fc    = nn.Identity()
    self.linear = nn.Linear(512, 1)
    self.encoder = nn.Sequential(
      self.backbone,
      self.linear
    ).to(device)

  def forward(self, X):
    return self.encoder(X)
  
model = ResNet18E(device)

optimizer   = torch.optim.Adam(model.parameters(), lr=lr,)
lossfn      = nn.MSELoss()

training_losses = []
validation_losses = []
r2s = []
for epoch in range(epochs):
  model.train()
  print(f'########################################')
  print(f'Epoch: {epoch+1}')
  losses = 0
  count  = 0
  for images, temperatures in train_loader:
    N = images.shape[0]
    V = images.shape[1]
    temperatures = temperatures.squeeze()
    ##
    ##
    ##
    pred  = model(images).squeeze()
    loss  = lossfn(pred, temperatures)

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses += loss.cpu().detach().numpy()
    count  += 1

    print(f'*', end='')
  print('\n')
  print(f'MSE : {losses / count}')
  print(f'RMSE: {np.sqrt(losses / count)}')
  training_losses.append(losses / count)

  model.eval()
  with torch.no_grad():
    losses = 0
    count  = 0
    y_pred = []
    y_test = []
    for images, temperatures in val_loader:
      N = images.shape[0]
      V = images.shape[1]
      images = images.to(device)
      temperatures = temperatures.to(device).squeeze()
      pred = model(images).squeeze()
      loss = lossfn(pred, temperatures)
      losses += loss.cpu().detach().numpy()
      count += 1

      y_pred.append(pred.squeeze().cpu().detach().numpy())
      y_test.append(temperatures.squeeze().cpu().detach().numpy())
      
  y_pred = np.array(y_pred).flatten()
  y_test = np.array(y_test).flatten()

  r2 = r2_score(y_test, y_pred)
  print(f'Validation: MSE : {losses / count}')
  print(f'Validation: RMSE: {np.sqrt(losses / count)}')
  print(f'Validation R2: {r2}')
  validation_losses.append(losses / count)
  r2s.append(r2)

Path to dataset files: C:\Users\tsope\.cache\kagglehub\datasets\juanda220485\synthetic-dataset-of-speckle-images\versions\1


c:\Users\tsope\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tsope\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


########################################
Epoch: 1
*****

MSE : 4702.7572265625
RMSE: 68.5766521971035
Validation: MSE : 4883.21875
Validation: RMSE: 69.88003112477841
Validation R2: -2.811937093734741
########################################
Epoch: 2
*****

MSE : 4544.34306640625
RMSE: 67.41174279312358
Validation: MSE : 4769.818359375
Validation: RMSE: 69.06387159271482
Validation R2: -2.7234151363372803
########################################
Epoch: 3
*****

MSE : 4227.318701171875
RMSE: 65.01783371638795
Validation: MSE : 4586.603515625
Validation: RMSE: 67.72446762895224
Validation R2: -2.5803937911987305
########################################
Epoch: 4
*****

MSE : 4187.21044921875
RMSE: 64.70865822452781
Validation: MSE : 4375.68798828125
Validation: RMSE: 66.1489832747356
Validation R2: -2.4157485961914062
########################################
Epoch: 5
*****

MSE : 4074.764794921875
RMSE: 63.83388437908095
Validation: MSE : 4180.21435546875
Validation: RMSE: 64.654577219781

In [4]:
from Utils import ReadRegressorData
num_epochs_r, training_losses, validation_losses, r2s = ReadRegressorData('Resnet18_400')
plt.figure(figsize=[15, 15])
plt.subplot(3, 1, 1)
plt.plot(num_epochs_r, np.array(training_losses), num_epochs_r, np.array(validation_losses))
plt.xlabel("Epochs")
plt.ylabel("Mean Squared Error")
plt.title("Mean Squared Error of Resnet18 + Regression Head")
plt.legend(['Training', 'Validation'])

plt.subplot(3, 1, 2)
plt.plot(num_epochs_r, np.sqrt(training_losses), num_epochs_r, np.sqrt(validation_losses))
plt.xlabel("Epochs")
plt.ylabel("Root Mean Squared Error")
plt.title("Root Mean Squared Error of Resnet18 + Regression Head")
plt.legend(['Training', 'Validation'])

plt.subplot(3, 1, 3)
plt.plot(num_epochs_r, np.array(r2s))
plt.xlabel("Epochs")
plt.ylabel("R2")
plt.title("Validation R2 of Resnet18 + Regression Head")

plt.tight_layout()
plt.show()

In [6]:
torch.save(model, 'resnet18e_400.pth')

In [7]:
np.save('train_losses.npy', np.array(training_losses))
np.save('val_losses.npy', np.array(validation_losses))
np.save('r2_score.npy', np.array(r2s))